# Copy and paste of the notebook's lab on FFNN w/ Addition of a Kaf Layer
* Taken from the course: "Neural Networks for Data Science Applications a.a. 19-20 Sapienza Univerisy of Rome"
* Accuracy improvements gained thanks to kafs
* Note, this notebook requires the SUSY dataset in the working folder (https://archive.ics.uci.edu/ml/datasets/SUSY)

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from kafnet import Kaf



In [2]:
# Sanity check 

#x = tf.random.uniform(shape=(3,28,28,3), minval=-20, maxval=40, dtype=tf.float32)
x = tf.random.uniform(shape=(3,10), minval=-20, maxval=40, dtype=tf.float32)

#kaf = Kaf(5, conv=True)
kaf = Kaf(5)
kaf(x)


<tf.Tensor: shape=(3, 10), dtype=float32, numpy=
array([[-3.4285966e-02,  8.5643893e-03, -9.2372652e-03,  1.2335321e-04,
         1.8697642e-03,  2.3586832e-02,  1.0291087e-04, -1.5569418e-02,
        -3.2498788e-02, -5.7054665e-02],
       [ 5.2916780e-03,  2.2996373e-02, -3.6069609e-02,  9.7880304e-02,
         2.5295720e-03,  8.3124135e-03,  2.7118027e-03, -1.3640668e-02,
         6.8371646e-06, -1.0347946e-01],
       [-2.4792650e-03,  3.0365984e-03, -1.3631162e-01, -2.1787807e-02,
        -1.6053287e-02,  1.7609477e-02,  4.0064398e-03,  6.0692959e-02,
        -1.7224330e-02, -3.5037516e-04]], dtype=float32)>

In [3]:
# Import SUSY Dataset used during the lab of NNfDS

import pandas as pd

susy = pd.read_csv('SUSY.csv', header=None)
X = susy.values[0:100000, 1:].astype(np.float32) # Note the casting to float32
y = susy.values[0:100000, 0:1]

train_susy = tf.data.Dataset.from_tensor_slices((X, y))

In [4]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential

def build_model():
  net = Sequential()
  net.add(Dense(50))
  net.add(Kaf(30))
  net.add(Dense(1, activation='sigmoid'))
  return net

In [5]:
net = build_model()

In [6]:
from tensorflow.keras import losses, optimizers, metrics

net.compile(optimizer=optimizers.Adam(), loss=losses.BinaryCrossentropy(), 
           metrics = [
               metrics.BinaryAccuracy(),
               metrics.AUC()
           ])

In [7]:
history = net.fit(train_susy.shuffle(1000).batch(64), epochs=3)

Train for 1563 steps
Epoch 1/3
1563/1563 [==============================] - 12s 8ms/step - loss: 0.4739 - binary_accuracy: 0.7762 - auc: 0.8457
Epoch 2/3
1563/1563 [==============================] - 13s 8ms/step - loss: 0.4438 - binary_accuracy: 0.7946 - auc: 0.8653
Epoch 3/3
1563/1563 [==============================] - 13s 8ms/step - loss: 0.4407 - binary_accuracy: 0.7958 - auc: 0.8671


In [8]:
net.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  950       
_________________________________________________________________
kaf_1 (Kaf)                  multiple                  1500      
_________________________________________________________________
dense_1 (Dense)              multiple                  51        
Total params: 2,501
Trainable params: 2,501
Non-trainable params: 0
_________________________________________________________________


In [9]:
# We use a separate set of 100k values
Xtest = susy.values[100000:200000, 1:].astype(np.float32)
ytest = susy.values[100000:200000, 0:1]

In [10]:
test_susy = tf.data.Dataset.from_tensor_slices((Xtest, ytest))

In [11]:
# We remove the last dimension from y
test_susy = tf.data.Dataset.from_tensor_slices((Xtest, 
                                                ytest.reshape(-1)))

In [12]:
# Better score than during the lab session!
net.evaluate(test_susy.batch(32))

3125/3125 [==============================] - 16s 5ms/step - loss: 0.4356 - binary_accuracy: 0.7997 - auc: 0.8711


[0.4355582052183151, 0.79975, 0.8710929]

In [2]:
x = tf.ones(shape=(1,5))

k = Kaf(10, ridge='elu')

In [3]:
a = k(x)

dioooo


In [4]:
a

<tf.Variable 'kaf/Variable:0' shape=(1, 5, 10) dtype=float32, numpy=
array([[[ -41.336376,  150.05258 , -311.37494 ,  470.4437  ,
         -567.94617 ,  564.80316 , -471.41086 ,  329.209   ,
         -175.44856 ,   63.158123],
        [ -41.336376,  150.05258 , -311.37494 ,  470.4437  ,
         -567.94617 ,  564.80316 , -471.41086 ,  329.209   ,
         -175.44856 ,   63.158123],
        [ -41.336376,  150.05258 , -311.37494 ,  470.4437  ,
         -567.94617 ,  564.80316 , -471.41086 ,  329.209   ,
         -175.44856 ,   63.158123],
        [ -41.336376,  150.05258 , -311.37494 ,  470.4437  ,
         -567.94617 ,  564.80316 , -471.41086 ,  329.209   ,
         -175.44856 ,   63.158123],
        [ -41.336376,  150.05258 , -311.37494 ,  470.4437  ,
         -567.94617 ,  564.80316 , -471.41086 ,  329.209   ,
         -175.44856 ,   63.158123]]], dtype=float32)>

In [23]:
d = tf.stack(kaf.dictionaryGen(10)[1])

d

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([-10.       ,  -7.7777777,  -5.5555553,  -3.3333333,  -1.1111112,
         1.1111112,   3.3333333,   5.5555553,   7.7777777,  10.       ],
      dtype=float32)>

In [27]:
def kernelMatrix(d, k_bwidth):
    
    """ d is supposed to be a one dimensional (n, ) tensor 
        return the kernel matrix K \in R^D*D where K_ij = ker(d_i, d_j) """

    d = tf.expand_dims(d, -1)
    print(d)
    return tf.exp(- k_bwidth * tf.square(d - tf.transpose(d)))

In [31]:


t = tf.keras.activations.tanh(d)
K = kernelMatrix(d, 0.6)
a = tf.linalg.inv(K + tf.eye(10)) * t

tf.Tensor(
[[-10.       ]
 [ -7.7777777]
 [ -5.5555553]
 [ -3.3333333]
 [ -1.1111112]
 [  1.1111112]
 [  3.3333333]
 [  5.5555553]
 [  7.7777777]
 [ 10.       ]], shape=(10, 1), dtype=float32)
